# Tugas Mengambil (Crawling Data) Tugas Akhir (pta.trunojoyo.ac.id)

Nama : Bihubbil Choir Aidifta<p>
NIM : 190411100121

# Instalasi Library
Pada tahap ini dilakukan instalasi library-library yang nantinya dibutuhkan untuk menjalankan program, berikut adalah library yang dibutuhkan : 

---



> ##  Scrapy
Scrapy merupakan kerangka kerja / framework sumber terbuka (open source) yang ditulis dengan bahasa pemrograman python dan berguna untuk melakukan ekstraksi data dari suatu web. Pada program ini kita akan menggunakan scrapy untuk mengambil/mengekstrak data abstrak dari website Portal Tugas Akhir Universitas Trunojoyo Madura.

> ## Pandas
Pandas merupakan perpustakaan / library sumber terbuka (Open Source) yang ditulis dengan bahasa pemrograman python dan berguna untuk melakukan analisis dan manipulasi data, khususnya untuk memanipulasi tabel numerik dan deret waktu.
Pada program ini kita akan memanfaatkan Pandas DataFrame untuk menempatkan data hasil crawling.

> ## Sastrawi
Sastrawi merupakan perpustakaan / library sumber terbuka yang ditulis dengan bahasa pemrograman python dan berguna untuk melakukan pengurangan kata-kata yang ter- infleksi dalam bahasa Indonesia ke bentuk baku-nya atau sesuai dengan standar kamus.


In [ ]:
%%capture
!pip install scrapy
!pip install pandas
!pip install Sastrawi

# Menghapus File lama
Karena disini kita akan menulis script dan mengekspor hasil ekstrak data ke sebuah berkas, maka untuk menghindari terjadinya duplikasi konten pada berkas tersebut, kita akan melakukan penghapusan berkas tersebut terlebih dahulu (jika berkas terebut ada)

In [ ]:
!rm -rf ptautm_scrape.py hasil.json && sleep 1

# Menulis Script Konfigurasi Scrapy
Pada tahap ini akan dibuat script kofigurasi scrapy, script ini sendiri berupa class yang berisi fungsi-fungsi untuk melakukan crawling dan scraping data abstrak yang berada pada website Portal Tugas Akhir Universitas Trunojoyo Madura.<p>
Berikut adalah penjelasan singkat mengenai script tersebut.

---
<br>

```
 start_urls = [
      'https://pta.trunojoyo.ac.id/c_search/byprod/10'
      ]
```

Variable tersebut merupakan variable yang berisi url awal yang nantinya akan di crawl oleh scrapy. Sebagai informasi, value yang berada pada variable tersebut merupakan url halaman yang berisi kumpulan tugas akhir dari Prodi Teknik Informatika.

---

<br>

```
def parse(self, response):
        for url in response.xpath('//li[contains(@data-id,"id-1")]/div[3]/a/@href'):
            yield response.follow(url, callback=self.parse_content)

        next_page = response.xpath('(//li/a[contains(@class, "pag_button")])[last() -1]/@href').get()
        if next_page != 'https://pta.trunojoyo.ac.id/c_search/byprod/10/20':
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)
```

Fungsi ini merupakan fungsi parsing url berguna untuk melakukan parsing/penguraian dari url awal guna untuk mendapatkan url dari masing-masing tugas akhir. Yang mana nantinya url dari masing-masing tugas akhir akan dilakukan pemanggilan fungsi parsing untuk mendapatkan konten dari masing-masing halaman tugas akhir tersebut.

Fungsi ini juga akan melakukan parsing url dari tombol next page yang berguna untuk mendapatkan url halaman selanjutnya yang juga berisi kumpulan tugas akhir pada prodi yang sama. Setelah dilakukan parsing, kemudian fungsi ini akan melakukan pemanggilan fungsi ini kembali, namun dengan mengganti awalan url dengan url hasil parsing dari tombol next page. 

Untuk batasannya, disini saya menggunakan halaman ke 21, sehingga url yang akan dicrawl adalah sebanyak 20 halaman (yang berisi kumpulan tugas akhir).



```
(if next_page != 'https://pta.trunojoyo.ac.id/c_search/byprod/10/21':)

```



---

<br>

```
def parse_content(self, response):
        
        item = {
            'judul' : response.xpath('//a[contains(@class, "title")]/text()').get(),
            'abstrak' : response.xpath('(//p[contains(@align, "justify")])[1]/text()').get()
        }

        yield item
```

Fungsi ini merupakan fungsi parsing konten yang berguna untuk melakukan parsing/penguraian dari dari url inputan guna untuk mendapatkan konten berupa teks (judul dan abstrak) dari suatu halaman tugas akhir.





In [ ]:
%%writefile -a ptautm_scrape.py
import string
import scrapy
from scrapy import Request

class ptaUTM(scrapy.Spider):
    name = "Abstract"
    start_urls = [
      'https://pta.trunojoyo.ac.id/c_search/byprod/10'
      ]
    def parse(self, response):
        for url in response.xpath('//li[contains(@data-id,"id-1")]/div[3]/a/@href'):
            yield response.follow(url, callback=self.parse_content)

        next_page = response.xpath('(//li/a[contains(@class, "pag_button")])[last() -1]/@href').get()
        if next_page != 'https://pta.trunojoyo.ac.id/c_search/byprod/10/21':
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_content(self, response):
        
        item = {
            'judul' : response.xpath('//a[contains(@class, "title")]/text()').get(),
            'abstrak' : response.xpath('(//p[contains(@align, "justify")])[1]/text()').get()
        }

        yield item

Writing ptautm_scrape.py


# Menjalankan Script Scrapy
Setelah sebelumnya kita menulis script konfigurasi scrapy, selanjutnya kita akan menjalankan script tersebut dengan perintah "scrapy runspider" yang diikuti dengan nama berkas script dan dan nama berkas output untuk menyimpan hasilnya.

In [ ]:
%%capture
!scrapy runspider ptautm_scrape.py -O hasil.json && sleep 2

# Import Hasil
Setelah perintah sebelumnya berhasil dijalankan, selanjutnya kita akan melakukan import isi berkas hasil scrape sebelumnya kedalam Pandas DataFrame.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_colwidth = None
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df = pd.read_json('hasil.json')
df

judul  \
0                                                                                                                                             APLIKASI KONTROL DAN MONITORING JARINGAN KOMPUTER BERBASIS MOBILE   
1                                                                                PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \r\nTERDISTRIBUSI MENGGUNAKAN ORACLE STUDI KASUS \r\nSIAKAD UNIVERSITAS TRUNOJOYO   
2                                                                                                            RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\nENKRIPSI KODE HTML MENGGUNAKAN\r\nALGORITMA BLOWFISH   
3                                                                                                                                       SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERAK MENGGUNAKAN FLARTOOLKIT   
4                                        SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALAN MATAKULIAH UNTUK PENGGUNAAN RUANGAN KULIAH BERSAMA DI UNIVERSITAS TRUNOJOYO MADURA MENGGUNAKAN ALGORITMA  \r\nGENETIKA   
5                                                                 Perancangan Sistem Informasi Badan Kepegawaian Daerah ( BKD ) Bangkalan Sebagai Sub sistem dari E-Government Bangkalan Menggunakan TOGAF ADM    
6                                                                                                      IMPLEMENTASI  ALGORITMA PRIM  DAN DEPTH FIRST SEARCH PADA PEMBUATAN MAZE GAME BERBASIS ANDROID OS MOBILE   
7                                                                                                                 EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEAR DISCRIMINANT ANALYSIS  UNTUK PENGENALAN WAJAH   
8                                                                                                                                     RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MENGGUNAKAN METODE FUZZY LOGIC   
9                                                                                                                                   Gerak Pekerja Pada Game Real Time Strategy Menggunakan Finite State Machine   
10                                                                                                                            DETEKSI COREPOINT SIDIK JARI MENGGUNAKAN METODE GEOMETRY OF REGION TECHNIQUE (GR)   
11                                                  RANCANG BANGUN APLIKASI PEMILIHAN TEKNIK REKAYASA KEBUTUHAN MENGGUNAKAN METODE SELF ORGANIZING MAP BERBASIS EUCLIDEAN DISTANCE DAN CANBERRA DISTANCE MATRIX   
12                                  SISTEM PENDUKUNG KEPUTUSAN REKOMENDASI MENU DIET BAGI PASIEN RAWAT INAP MENGGUNAKAN METODE HARRIS BENEDICT DAN EUCLIDEAN (Studi Kasus : RSUD Dr. H. Moh. Anwar Sumenep)\r\n   
13                                                                    APLIKASI METODE FUZZY ANALYTIC NETWORK PROCESS (FANP) UNTUK MENDUKUNG KEPUTUSAN PROSES PROMOSI JABATAN DI PT. SURYA MADISTRINDO PAMEKASAN   
14                                                    PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM INFORMASI DINAS PERINDUSTRIAN & PERDAGANGAN SEBAGAI SUB SISTEM ARSITEKTUR E-GOVERNMENT KABUPATEN BANGKALAN   
15                                                                                                                  ONTOLOGY SEMANTIC WEB UNTUK PENCARIAN TUGAS AKHIR PADA SISTEM INFORMASI TUGAS AKHIR(SIMTAK)   
16                                                                              SISTEM PEROLEHAN CITRA BERBASIS ISI MENGGUNAKAN\r\nGRAY LEVEL DIFFERENCE METHOD BERDASARKAN\r\nCIRI TEKSTUR PADA POLA BATIK\r\n   
17                                                                          PENGENALAN TULISAN TANGAN HURUF ALFABET\r\nDENGAN METODE MODIFIED DIRECTION FEATURE (MDF)\r\nDAN LEARNING VECTOR QUANTIZATION (LVQ)   
18                                                                                  PENGENALAN POLA SENYUMAN BERBASIS EKSTRAKSI FITUR PRINCIPAL COMPONENT ANALYSIS (PCA) DAN LINIER DESCRIMINANT 

# Preprocessing Data Abstrak

## Case Folding
Case folding merupakan tahap text preprocessing yang berguna untuk :


*   Mengubah huruf kapital menjadi huruf kecil
*   Mengapus tanda baca
*   Menghapus angka
*   Menghapus karakter kosong



In [ ]:
import string

#mengubah menjadi huruf kecil
df['abstrak'] = df['abstrak'].str.lower()

#menghapus tanda baca
for char in string.punctuation:
    df['abstrak'] = df['abstrak'].replace(r'[\%s]'%char," ", regex=True)

#menghapus angka
df['abstrak'] = df['abstrak'].replace(r'\d+',' ', regex=True)

#menghapus karakter kosong
df['abstrak'] = df['abstrak'].replace(r'\s+',' ', regex=True)
df['abstrak'][3]



'seiring perkembangan teknologi yang ada didunia muncul teknologi yang disebut augmented reality augmented reality ar merupakan upaya untuk menggabungkan dunia maya dan dunia virtual yang dibuat melalui komputer sehingga batas antara keduanya menjadi sangat tipis head mounted display hmd merupakan contoh hasil dari penelitian tentang augmented reality ini merupakan satu satunya peralatan dasar dalam teknologi teknologi terbaru seiring berjalannya waktu augmented reality berkembang pesat sehingga memungkinkan pengembangan aplikasi ini di berbagai bidang untuk mengembangkan ar banyak sekali library pendukung yang dapat digunakan flartoolkit adalah salah satunya flartoolkit merupakan library pendukung augmented reality pada platform flash berdasarkan sistem ar animasi benda bergerak yang dibuat adalah animasi bergerak permainan pingpong dimana telah dikolaborasikan dengan augmented reality untuk menggerakkan setiap langkah dari playernya interaksi antar obyek tersebut dapat dilihat dari g

## Stopwords Removal
StopWords Removal merupakan tahap prepocessing yang berguna untuk menhapus kata umum yang biasanya muncul dalam jumlah besar dan dianggap tidak memiliki makna. Contoh stopword dalam bahasa Indonesia adalah “yang”, “dan”, “di”, “dari”, dll.


In [ ]:
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/smilesense/stopwords-id/master/stopwords-id.json") as list_stopwords:
    data_stopword = json.load(list_stopwords)

for i in data_stopword:
    df['abstrak'] = df['abstrak'].replace(r'\b%s\b'%i, '', regex=True)
df['abstrak'] = df['abstrak'].replace(r'\s+',' ', regex=True)

df['abstrak'][3]

'seiring perkembangan teknologi didunia muncul teknologi augmented reality augmented reality ar upaya menggabungkan dunia maya dunia virtual komputer batas tipis head mounted display hmd contoh hasil penelitian augmented reality satunya peralatan dasar teknologi teknologi terbaru seiring berjalannya augmented reality berkembang pesat pengembangan aplikasi bidang mengembangkan ar library pendukung flartoolkit salah satunya flartoolkit library pendukung augmented reality platform flash berdasarkan sistem ar animasi benda bergerak animasi bergerak permainan pingpong dimana dikolaborasikan augmented reality menggerakkan langkah playernya interaksi obyek gerak bola dipukul obyek virtual ditampilkan augmented reality '

## Stemming
Stemming merupakan tahapan dalam preprocessing yang berguna untuk mengubah kata yang memiliki imbuhan menjadi kata dasarnya. Contohnya adalah, misal pada suatu teks terdapat kata "berjalan", maka dengan menerapkan metode stemming, teks tersebut akan diubah menjadi kata "jalan" atau bentuk dasar dari kata "berjalan". 

In [ ]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process

ind = 0
for sentence in df['abstrak']:
    df['abstrak'][ind] = stemmer.stem(str(sentence))
    ind+=1
df

judul  \
0                                                                                                                                             APLIKASI KONTROL DAN MONITORING JARINGAN KOMPUTER BERBASIS MOBILE   
1                                                                                PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \r\nTERDISTRIBUSI MENGGUNAKAN ORACLE STUDI KASUS \r\nSIAKAD UNIVERSITAS TRUNOJOYO   
2                                                                                                            RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\nENKRIPSI KODE HTML MENGGUNAKAN\r\nALGORITMA BLOWFISH   
3                                                                                                                                       SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERAK MENGGUNAKAN FLARTOOLKIT   
4                                        SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALAN MATAKULIAH UNTUK PENGGUNAAN RUANGAN KULIAH BERSAMA DI UNIVERSITAS TRUNOJOYO MADURA MENGGUNAKAN ALGORITMA  \r\nGENETIKA   
5                                                                 Perancangan Sistem Informasi Badan Kepegawaian Daerah ( BKD ) Bangkalan Sebagai Sub sistem dari E-Government Bangkalan Menggunakan TOGAF ADM    
6                                                                                                      IMPLEMENTASI  ALGORITMA PRIM  DAN DEPTH FIRST SEARCH PADA PEMBUATAN MAZE GAME BERBASIS ANDROID OS MOBILE   
7                                                                                                                 EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEAR DISCRIMINANT ANALYSIS  UNTUK PENGENALAN WAJAH   
8                                                                                                                                     RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MENGGUNAKAN METODE FUZZY LOGIC   
9                                                                                                                                   Gerak Pekerja Pada Game Real Time Strategy Menggunakan Finite State Machine   
10                                                                                                                            DETEKSI COREPOINT SIDIK JARI MENGGUNAKAN METODE GEOMETRY OF REGION TECHNIQUE (GR)   
11                                                  RANCANG BANGUN APLIKASI PEMILIHAN TEKNIK REKAYASA KEBUTUHAN MENGGUNAKAN METODE SELF ORGANIZING MAP BERBASIS EUCLIDEAN DISTANCE DAN CANBERRA DISTANCE MATRIX   
12                                  SISTEM PENDUKUNG KEPUTUSAN REKOMENDASI MENU DIET BAGI PASIEN RAWAT INAP MENGGUNAKAN METODE HARRIS BENEDICT DAN EUCLIDEAN (Studi Kasus : RSUD Dr. H. Moh. Anwar Sumenep)\r\n   
13                                                                    APLIKASI METODE FUZZY ANALYTIC NETWORK PROCESS (FANP) UNTUK MENDUKUNG KEPUTUSAN PROSES PROMOSI JABATAN DI PT. SURYA MADISTRINDO PAMEKASAN   
14                                                    PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM INFORMASI DINAS PERINDUSTRIAN & PERDAGANGAN SEBAGAI SUB SISTEM ARSITEKTUR E-GOVERNMENT KABUPATEN BANGKALAN   
15                                                                                                                  ONTOLOGY SEMANTIC WEB UNTUK PENCARIAN TUGAS AKHIR PADA SISTEM INFORMASI TUGAS AKHIR(SIMTAK)   
16                                                                              SISTEM PEROLEHAN CITRA BERBASIS ISI MENGGUNAKAN\r\nGRAY LEVEL DIFFERENCE METHOD BERDASARKAN\r\nCIRI TEKSTUR PADA POLA BATIK\r\n   
17                                                                          PENGENALAN TULISAN TANGAN HURUF ALFABET\r\nDENGAN METODE MODIFIED DIRECTION FEATURE (MDF)\r\nDAN LEARNING VECTOR QUANTIZATION (LVQ)   
18                                                                                  PENGENALAN POLA SENYUMAN BERBASIS EKSTRAKSI FITUR PRINCIPAL COMPONENT ANALYSIS (PCA) DAN LINIER DESCRIMINANT 

## Tokenisasi

Tokenizing adalah tahap preprocessing yang berguna untuk memisahkan teks menjadi potongan-potongan yang disebut sebagai token untuk kemudian di analisa. Kata, angka, simbol, tanda baca dan entitas penting lainnya dapat dianggap sebagai token.

In [ ]:
indeks = 0
df2 = df.copy()
for abstrak in df2['abstrak']:
    df2['abstrak'][indeks] = str(abstrak).split()
    indeks+=1

df2['abstrak'][3]


['iring',
 'kembang',
 'teknologi',
 'dunia',
 'muncul',
 'teknologi',
 'augmented',
 'reality',
 'augmented',
 'reality',
 'ar',
 'upaya',
 'gabung',
 'dunia',
 'maya',
 'dunia',
 'virtual',
 'komputer',
 'batas',
 'tipis',
 'head',
 'mounted',
 'display',
 'hmd',
 'contoh',
 'hasil',
 'teliti',
 'augmented',
 'reality',
 'satu',
 'alat',
 'dasar',
 'teknologi',
 'teknologi',
 'baru',
 'iring',
 'jalan',
 'augmented',
 'reality',
 'kembang',
 'pesat',
 'kembang',
 'aplikasi',
 'bidang',
 'kembang',
 'ar',
 'library',
 'dukung',
 'flartoolkit',
 'salah',
 'satu',
 'flartoolkit',
 'library',
 'dukung',
 'augmented',
 'reality',
 'platform',
 'flash',
 'dasar',
 'sistem',
 'ar',
 'animasi',
 'benda',
 'gerak',
 'animasi',
 'gerak',
 'main',
 'pingpong',
 'mana',
 'kolaborasi',
 'augmented',
 'reality',
 'gerak',
 'langkah',
 'playernya',
 'interaksi',
 'obyek',
 'gerak',
 'bola',
 'pukul',
 'obyek',
 'virtual',
 'tampil',
 'augmented',
 'reality']

## Hasil Preprocessing


In [ ]:
df2

judul  \
0                                                                                                                                             APLIKASI KONTROL DAN MONITORING JARINGAN KOMPUTER BERBASIS MOBILE   
1                                                                                PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \r\nTERDISTRIBUSI MENGGUNAKAN ORACLE STUDI KASUS \r\nSIAKAD UNIVERSITAS TRUNOJOYO   
2                                                                                                            RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\nENKRIPSI KODE HTML MENGGUNAKAN\r\nALGORITMA BLOWFISH   
3                                                                                                                                       SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERAK MENGGUNAKAN FLARTOOLKIT   
4                                        SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALAN MATAKULIAH UNTUK PENGGUNAAN RUANGAN KULIAH BERSAMA DI UNIVERSITAS TRUNOJOYO MADURA MENGGUNAKAN ALGORITMA  \r\nGENETIKA   
5                                                                 Perancangan Sistem Informasi Badan Kepegawaian Daerah ( BKD ) Bangkalan Sebagai Sub sistem dari E-Government Bangkalan Menggunakan TOGAF ADM    
6                                                                                                      IMPLEMENTASI  ALGORITMA PRIM  DAN DEPTH FIRST SEARCH PADA PEMBUATAN MAZE GAME BERBASIS ANDROID OS MOBILE   
7                                                                                                                 EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEAR DISCRIMINANT ANALYSIS  UNTUK PENGENALAN WAJAH   
8                                                                                                                                     RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MENGGUNAKAN METODE FUZZY LOGIC   
9                                                                                                                                   Gerak Pekerja Pada Game Real Time Strategy Menggunakan Finite State Machine   
10                                                                                                                            DETEKSI COREPOINT SIDIK JARI MENGGUNAKAN METODE GEOMETRY OF REGION TECHNIQUE (GR)   
11                                                  RANCANG BANGUN APLIKASI PEMILIHAN TEKNIK REKAYASA KEBUTUHAN MENGGUNAKAN METODE SELF ORGANIZING MAP BERBASIS EUCLIDEAN DISTANCE DAN CANBERRA DISTANCE MATRIX   
12                                  SISTEM PENDUKUNG KEPUTUSAN REKOMENDASI MENU DIET BAGI PASIEN RAWAT INAP MENGGUNAKAN METODE HARRIS BENEDICT DAN EUCLIDEAN (Studi Kasus : RSUD Dr. H. Moh. Anwar Sumenep)\r\n   
13                                                                    APLIKASI METODE FUZZY ANALYTIC NETWORK PROCESS (FANP) UNTUK MENDUKUNG KEPUTUSAN PROSES PROMOSI JABATAN DI PT. SURYA MADISTRINDO PAMEKASAN   
14                                                    PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM INFORMASI DINAS PERINDUSTRIAN & PERDAGANGAN SEBAGAI SUB SISTEM ARSITEKTUR E-GOVERNMENT KABUPATEN BANGKALAN   
15                                                                                                                  ONTOLOGY SEMANTIC WEB UNTUK PENCARIAN TUGAS AKHIR PADA SISTEM INFORMASI TUGAS AKHIR(SIMTAK)   
16                                                                              SISTEM PEROLEHAN CITRA BERBASIS ISI MENGGUNAKAN\r\nGRAY LEVEL DIFFERENCE METHOD BERDASARKAN\r\nCIRI TEKSTUR PADA POLA BATIK\r\n   
17                                                                          PENGENALAN TULISAN TANGAN HURUF ALFABET\r\nDENGAN METODE MODIFIED DIRECTION FEATURE (MDF)\r\nDAN LEARNING VECTOR QUANTIZATION (LVQ)   
18                                                                                  PENGENALAN POLA SENYUMAN BERBASIS EKSTRAKSI FITUR PRINCIPAL COMPONENT ANALYSIS (PCA) DAN LINIER DESCRIMINANT 

# Term Frequency (TF)
Term Frequency merupakan metode yang digunakan untuk mengetahui seberapa sering suatu kata muncul dalam suatu teks. Semakin banyak frekuensi kemunculan dari kata tsb, semakin besar pula nanti nilainya.



## Term Frequency Keseluruhan

In [ ]:
listkata = []
for abstrak in df['abstrak']:
    listkata = listkata + str(abstrak).split()

listkata_nodup = list(dict.fromkeys(listkata))
hasil_hitung = {}
for test1 in listkata_nodup:
    jumlah = 0
    for test2 in range(len(listkata)):
        if test1 == listkata[test2]:
            jumlah+=1
    hasil_hitung.update({'%s'%test1 : jumlah})
    
hasil_hitung = dict(sorted(hasil_hitung.items(), key=lambda item: item[1], reverse=True))
print(hasil_hitung)

{'hasil': 177, 'sistem': 160, 'metode': 156, 'proses': 136, 'data': 127, 'citra': 118, 'nilai': 113, 'aplikasi': 100, 'teliti': 98, 'uji': 76, 'game': 65, 'milik': 64, 'butuh': 63, 'salah': 62, 'tentu': 62, 'kenal': 59, 'kembang': 58, 'ajar': 58, 'coba': 57, 'kriteria': 57, 'kunci': 56, 'dasar': 56, 'tingkat': 53, 'akurasi': 51, 'usaha': 51, 'algoritma': 50, 'oleh': 49, 'informasi': 48, 'putus': 48, 'cari': 47, 'teknologi': 45, 'jalan': 41, 'sesuai': 40, 'tulis': 40, 'guna': 39, 'dukung': 39, 'pilih': 38, 'hitung': 36, 'tangan': 35, 'deteksi': 35, 'bahasa': 34, 'z': 34, 'bentuk': 33, 'tuju': 33, 'kerja': 33, 'bangun': 33, 'mudah': 31, 'dokumen': 31, 'terap': 31, 'amal': 31, 'madura': 30, 'batik': 30, 'wajah': 29, 'fitur': 29, 'skenario': 29, 'bas': 27, 'web': 27, 'banding': 27, 'jaring': 26, 'ambil': 26, 'klasifikasi': 26, 'fuzzy': 26, 'teknik': 26, 'beasiswa': 26, 'tugas': 25, 'dapat': 25, 'bangkal': 25, 'pola': 25, 'didik': 25, 'produksi': 25, 'bantu': 23, 'manusia': 23, 'ukur': 23, 

## Term Frequency Tiap Abstrak

In [ ]:
# for k in hasil_hitung.copy():
#     if hasil_hitung[k] < 10:
#         del hasil_hitung[k]

def terms(dataframe):
  pd.options.mode.chained_assignment = None
  pd.options.display.max_colwidth = None
  pd.options.display.max_columns = None
  pd.options.display.max_rows = None

  df3 = dataframe.copy()
  print("Masukkan Jumlah Terms yang ingin digunakan (max %s)"%len(hasil_hitung))
  max_term = int(input())
  if max_term > len(hasil_hitung):
    max_term = len(hasil_hitung)
  a = 1
  for inter in range(len(df3['abstrak'])):
      for fitur in hasil_hitung:
          df3['term-%s (%s)'%(a,fitur)] = 0

          if a >= max_term:
            a = 1
            break
          a+=1

  for inter in range(len(df3['abstrak'])):
    for fitur in hasil_hitung:
        cek = df3['abstrak'][inter]
        jumlah = 0
        for iter2 in range(len(cek)):
          if fitur == cek[iter2]:
            jumlah+=1
        df3['term-%s (%s)'%(a,fitur)][inter] = jumlah
        if a >= max_term:
          a = 1
          break
        a+=1
  df3.to_csv(r'my_data.csv', index=False)
  return df3
  

terms(df2)

Masukkan Jumlah Terms yang ingin digunakan (max 1912)
500


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


judul  \
0                                                                                                                                             APLIKASI KONTROL DAN MONITORING JARINGAN KOMPUTER BERBASIS MOBILE   
1                                                                                PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \r\nTERDISTRIBUSI MENGGUNAKAN ORACLE STUDI KASUS \r\nSIAKAD UNIVERSITAS TRUNOJOYO   
2                                                                                                            RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\nENKRIPSI KODE HTML MENGGUNAKAN\r\nALGORITMA BLOWFISH   
3                                                                                                                                       SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERAK MENGGUNAKAN FLARTOOLKIT   
4                                        SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALAN MATAKULIAH UNTUK PENGGUNAAN RUANGAN KULIAH BERSAMA DI UNIVERSITAS TRUNOJOYO MADURA MENGGUNAKAN ALGORITMA  \r\nGENETIKA   
5                                                                 Perancangan Sistem Informasi Badan Kepegawaian Daerah ( BKD ) Bangkalan Sebagai Sub sistem dari E-Government Bangkalan Menggunakan TOGAF ADM    
6                                                                                                      IMPLEMENTASI  ALGORITMA PRIM  DAN DEPTH FIRST SEARCH PADA PEMBUATAN MAZE GAME BERBASIS ANDROID OS MOBILE   
7                                                                                                                 EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEAR DISCRIMINANT ANALYSIS  UNTUK PENGENALAN WAJAH   
8                                                                                                                                     RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MENGGUNAKAN METODE FUZZY LOGIC   
9                                                                                                                                   Gerak Pekerja Pada Game Real Time Strategy Menggunakan Finite State Machine   
10                                                                                                                            DETEKSI COREPOINT SIDIK JARI MENGGUNAKAN METODE GEOMETRY OF REGION TECHNIQUE (GR)   
11                                                  RANCANG BANGUN APLIKASI PEMILIHAN TEKNIK REKAYASA KEBUTUHAN MENGGUNAKAN METODE SELF ORGANIZING MAP BERBASIS EUCLIDEAN DISTANCE DAN CANBERRA DISTANCE MATRIX   
12                                  SISTEM PENDUKUNG KEPUTUSAN REKOMENDASI MENU DIET BAGI PASIEN RAWAT INAP MENGGUNAKAN METODE HARRIS BENEDICT DAN EUCLIDEAN (Studi Kasus : RSUD Dr. H. Moh. Anwar Sumenep)\r\n   
13                                                                    APLIKASI METODE FUZZY ANALYTIC NETWORK PROCESS (FANP) UNTUK MENDUKUNG KEPUTUSAN PROSES PROMOSI JABATAN DI PT. SURYA MADISTRINDO PAMEKASAN   
14                                                    PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM INFORMASI DINAS PERINDUSTRIAN & PERDAGANGAN SEBAGAI SUB SISTEM ARSITEKTUR E-GOVERNMENT KABUPATEN BANGKALAN   
15                                                                                                                  ONTOLOGY SEMANTIC WEB UNTUK PENCARIAN TUGAS AKHIR PADA SISTEM INFORMASI TUGAS AKHIR(SIMTAK)   
16                                                                              SISTEM PEROLEHAN CITRA BERBASIS ISI MENGGUNAKAN\r\nGRAY LEVEL DIFFERENCE METHOD BERDASARKAN\r\nCIRI TEKSTUR PADA POLA BATIK\r\n   
17                                                                          PENGENALAN TULISAN TANGAN HURUF ALFABET\r\nDENGAN METODE MODIFIED DIRECTION FEATURE (MDF)\r\nDAN LEARNING VECTOR QUANTIZATION (LVQ)   
18                                                                                  PENGENALAN POLA SENYUMAN BERBASIS EKSTRAKSI FITUR PRINCIPAL COMPONENT ANALYSIS (PCA) DAN LINIER DESCRIMINANT 